# Initialization

## Import libraries

In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import ssl # Quickfix to torchaudio ssl error
ssl._create_default_https_context = ssl._create_unverified_context

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

/Users/belatijagadbintangsyuhada/.pyenv/versions/3.11.9/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Preprocessing

In [2]:
class DiabeticDataset(Dataset):
    def __init__(self, root_dir: str, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        
        for label in os.listdir(root_dir):
            label_dir = os.path.join(root_dir, label)
            if not os.path.isdir(label_dir):
                continue
            for img_name in os.listdir(label_dir):
                if 'L' in img_name:
                    left_img_path = os.path.join(label_dir, img_name)
                    right_img_name = img_name.replace('L', 'R')
                    right_img_path = os.path.join(label_dir, right_img_name)
                    if os.path.exists(right_img_path):
                        self.data.append((left_img_path, right_img_path, label))
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        left_img_path, right_img_path, label = self.data[idx]
        left_image = cv2.imread(left_img_path)
        right_image = cv2.imread(right_img_path)

        if self.transform:
            left_image = self.transform(left_image)
            right_image = self.transform(right_image)
        
        label = 1 if label == 'diabetic' else 0
        
        return left_image, right_image, label

In [3]:
def preprocessing(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = cv2.resize(image, (224, 224))  # Resize to 224x224
    image = image / 255.0  # Normalize to [0, 1]
    image = np.transpose(image, (2, 0, 1))  # Convert to (C, H, W)
    image = torch.tensor(image, dtype=torch.float32)
    return image

In [4]:
root_dir = './images/train'

train_dataset = DiabeticDataset(root_dir, transform=preprocessing)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

In [5]:
root_dir = './images/val'

val_dataset = DiabeticDataset(root_dir, transform=preprocessing)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
# Get a batch of data
data_iter = iter(train_dataloader)
left_images, right_images, labels = next(data_iter)

# Choose an index to display
index = 0

# Convert tensor to image format
left_image = left_images[index]
right_image = right_images[index]

# Convert tensor to numpy array and denormalize if needed
left_image_np = left_image.numpy().transpose((0, 1, 2))
right_image_np = right_image.numpy().transpose((0, 1, 2))

# Display the images
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Left Foot")
plt.imshow(left_image_np)

plt.subplot(1, 2, 2)
plt.title("Right Foot")
plt.imshow(right_image_np)

plt.show()

# Modeling

## Model construction

In [109]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        # self.dropout = nn.Dropout2d(p=.1)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        # x = self.dropout(x)
        x = F.gelu(x)
        return F.max_pool2d(x, 2)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv_block_left = nn.Sequential(
            ConvBlock(3, 32),
            ConvBlock(32, 64),
            ConvBlock(64, 128),
            ConvBlock(128, 256),
            ConvBlock(256, 512),
        )
        self.conv_block_right = nn.Sequential(
            ConvBlock(3, 32),
            ConvBlock(32, 64),
            ConvBlock(64, 128),
            ConvBlock(128, 256),
            ConvBlock(256, 512),
        )
        self.fc1 = nn.Linear(512 * 7 * 7 * 2, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, left_image, right_image):
        x_left = self.conv_block_left(left_image)
        x_right = self.conv_block_right(right_image)
        
        x_left = x_left.reshape(x_left.size(0), -1)
        x_right = x_right.reshape(x_right.size(0), -1)
        
        x = torch.cat((x_left, x_right), dim=1)
        
        x = F.gelu(self.fc1(x))
        x = self.fc2(x)
        
        return x

# Training and Validation Loop

In [12]:
def training_loop(model, epochs, optimizer, loss_fn, data):
    for t in range(epochs):
        loop = tqdm(data, total=len(data))
        model.train()

        for _, (left, right, y) in enumerate(loop):
            optimizer.zero_grad()

            left, right, y = left.to(device), right.to(device), y.to(device)
            
            y = y.unsqueeze(1).float() 
            pred = model(left, right)
            loss = loss_fn(pred, y)
            
            loss.backward()
            optimizer.step()

            loop.set_description(f"Epoch [{t+1}/{epochs}]")
            loop.set_postfix(loss=loss.item())

    print("Training completed.")

def validation_loop(model, loss_fn, data_loader, device):
    model.eval()
    size = len(data_loader.dataset)
    num_batches = len(data_loader)
    test_loss, correct = 0.0, 0

    with torch.no_grad():
        for left, right, y in data_loader:
            left, right, y = left.to(device), right.to(device), y.to(device)
            
            # Forward pass
            pred = model(left, right)
            
            # Calculate loss
            loss = loss_fn(pred, y.unsqueeze(1).float())
            test_loss += loss.item()
            
            # Calculate number of correct predictions
            pred_labels = (pred > 0.5).float()  # Thresholding for binary classification
            correct += (pred_labels.squeeze() == y).sum().item()
        
    # Average loss and accuracy
    test_loss /= num_batches
    accuracy = (correct / size) * 100

    print(f"Validation Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [110]:
model = Model().to(device)

epochs = 35
optimizer = AdamW(params=model.parameters())
loss_fn = nn.BCEWithLogitsLoss()

training_loop(model, epochs, optimizer, loss_fn, train_dataloader)

validation_loop(model, loss_fn, val_dataloader, device)

Epoch [35/35]: 100%|██████████| 4/4 [00:01<00:00,  3.03it/s, loss=0.0875]

Training completed.
Validation Error: 
 Accuracy: 79.5%, Avg loss: 0.339909



In [76]:
sum(p.numel() for p in model.parameters())

28832257